**Задание 1**  
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
# создаем датафрейм с количеством оценок каждого пользователя (+сортируем, +оставляем один столбец, +сбрасываем индексы)

quantity_df = df.groupby('userId').count().sort_values('timestamp', ascending=False)[['timestamp']].reset_index()
quantity_df.head()

,userId,timestamp
0,547,2391
1,564,1868
2,624,1735
3,15,1700
4,73,1610


In [4]:
# переименовываем столбец timestamp в quantity

quantity_df.columns = ['userId', 'quantity']
# можно и так:
# quantity_df.rename(columns={'timestamp': 'quantity'}, inplace=True)
quantity_df.head()

,userId,quantity
0,547,2391
1,564,1868
2,624,1735
3,15,1700
4,73,1610


In [5]:
# объединяем исходный датафрейм с тем, в котором посчитано количество оценок каждого пользователя

full_df = df.merge(quantity_df, how='left')
full_df.head()

,userId,movieId,rating,timestamp,quantity
0,1,31,2.5,1260759144,20
1,1,1029,3.0,1260759179,20
2,1,1061,3.0,1260759182,20
3,1,1129,2.0,1260759185,20
4,1,1172,4.0,1260759205,20


In [6]:
# фильтруем пользователей по количеству оценок (только больше 100)

cut_df = full_df[ full_df.quantity > 100 ]
cut_df.head()

,userId,movieId,rating,timestamp,quantity
147,4,10,4.0,949810645,204
148,4,34,5.0,949919556,204
149,4,112,5.0,949810582,204
150,4,141,5.0,949919681,204
151,4,153,4.0,949811346,204


In [7]:
# применяем к полученному датафрейму (users >100 оценок) функцию, определяющую среднее "время жизни" пользователя в днях

cut_df.groupby('userId').apply(lambda x: (x.timestamp.max() - x.timestamp.min())/60/60/24).head(10)

userId
4        2.356019
8        0.985961
15    5455.943241
17       0.093206
19       0.061134
21      15.803611
22       1.058924
23     212.293160
26     234.574780
30    1340.882824
dtype: float64

**Задание 2**  
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [11]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [12]:
# первая таблица

rzd.merge(auto, how='outer').merge(air, how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [13]:
# вторая таблица

client_base.merge(rzd, how='outer').merge(auto, how='outer').merge(air, how='outer')

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**  
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

**Ответ**  
Информацию о местоположении пользователя можно было бы представить в виде датафрейма, включающего в себя  
* id пользователя,
* значения Unix time (в момент "когда они требовались приложению", т.е. в момент визитов и/или покупок)
* и собственно координаты его местоположения в этот момент времени.  

Вероятно, у визитов и покупок из основного датафрейма с прочей информацией также есть данные о времени. Соответственно, два датафрейма можно было бы увязать между собой через Unix time с помощью merge и иметь информацию о местополодении пользователя в момент совершения визита или покупки, поскольку для отдельного пользователя каждый timestamp уникален.  
Не до конца понятен второй вопрос. Если речь о времени, и оно действительно известно, то связь происходит через него, и факт удаленности координат друг от друга не играет особой роли.  
Опять же, нет уверенности в том, что третий вопрос понят верно, но если речь идет о координатах пользователя в определенное время и пользе, которую дает эта информация, то из этого многое можно вынести для приложений с рекомендательными системами. Скажем, они могут предложить пользователю воспользоваться ближайшим магазином, кафе, заправкой, показать погоду именно в той части города, в которой он находится, выдать соответствующую рекламу и многое другое.  

Если неправильно понял задание, прошу меня простить ¯\\_(ツ)_/¯